In [1]:
import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml

In [2]:
# HAMILTONIAN
nqubits = 4
coeffs = np.ones(nqubits)
obs = [qml.PauliZ(i) for i in range(nqubits)]
hamiltonian = qml.Hamiltonian(coeffs, obs)
print(hamiltonian)

  (1.0) [Z0]
+ (1.0) [Z1]
+ (1.0) [Z2]
+ (1.0) [Z3]


In [3]:
# INITIAL POINT
shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=nqubits)
params_0 = np.random.random(size=shape)
print(params_0)

[[[0.97252485 0.45419912 0.26444932]
  [0.5206713  0.43298856 0.22681604]
  [0.53680998 0.88728903 0.97656399]
  [0.70757694 0.83381451 0.79270477]]

 [[0.67121618 0.064187   0.15992042]
  [0.88146502 0.78491736 0.02305105]
  [0.6171727  0.16132    0.52272394]
  [0.15353503 0.41538511 0.31865611]]]


In [6]:
# ANSTAZ AND COST
dev = qml.device('default.qubit', wires=nqubits)

@qml.qnode(dev)
def cost(parameters):
    qml.StronglyEntanglingLayers(weights=parameters, wires=range(nqubits))
    return qml.expval(hamiltonian)

print(qml.draw(cost, expansion_strategy="device")(params_0))
print(params_0.shape)

0: ──Rot(0.97,0.45,0.26)─╭●───────╭X──Rot(0.67,0.06,0.16)─╭●────╭X────┤ ╭<𝓗>
1: ──Rot(0.52,0.43,0.23)─╰X─╭●────│───Rot(0.88,0.78,0.02)─│──╭●─│──╭X─┤ ├<𝓗>
2: ──Rot(0.54,0.89,0.98)────╰X─╭●─│───Rot(0.62,0.16,0.52)─╰X─│──╰●─│──┤ ├<𝓗>
3: ──Rot(0.71,0.83,0.79)───────╰X─╰●──Rot(0.15,0.42,0.32)────╰X────╰●─┤ ╰<𝓗>
(2, 4, 3)


TypeError: compute_decomposition() missing 4 required positional arguments: 'weights', 'wires', 'ranges', and 'imprimitive'

In [12]:
# OPTIMIZATION PARAMETERS
max_iterations = 100
step_size = 0.5
conv_tol = 1e-05
epsilon = .01
# OPTIMIZER
opt = qml.GradientDescentOptimizer(step_size)
params = params_0
gd_cost = []
for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost, params)
    params = params + np.random.normal(loc=0, scale=epsilon, size=shape)
    gd_cost.append(prev_energy)
    energy = cost(params)
    conv = np.abs(energy - prev_energy)
    if n % 20 == 0: print("Iteration = {:},  Energy = {:.8f} Ha".format(n, energy))
    if conv <= conv_tol: break

Iteration = 0,  Energy = 0.01576314 Ha
Iteration = 20,  Energy = -3.99535033 Ha
Iteration = 40,  Energy = -3.99538953 Ha
Iteration = 60,  Energy = -3.98967315 Ha
Iteration = 80,  Energy = -3.99682487 Ha


In [10]:
opt = qml.QNGOptimizer(step_size, lam=0.001, approx="block-diag")
params = params_0
prev_energy = cost(params)
qngd_cost = []
for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost, params)
    params = params + np.random.normal(loc=0, scale=epsilon, size=len(params))
    qngd_cost.append(prev_energy)
    energy = cost(params)
    conv = np.abs(energy - prev_energy)
    if n % 4 == 0: print("Iteration = {:},  Energy = {:.8f} Ha".format(n, energy))
    if conv <= conv_tol: break

Iteration = 0,  Energy = -0.32164519 Ha
Iteration = 4,  Energy = -0.46875033 Ha
Iteration = 8,  Energy = -0.85091050 Ha
Iteration = 12,  Energy = -1.13575339 Ha
Iteration = 16,  Energy = -1.13618916 Ha
